# Preprocesamiento

Para este apartado lo que haremos será hacer 3 tipos de preprocesamiento:

- Escalar las imágenes a 300x300
- Aumentar datos (data augmentation)
- Normalizar de 0 a 1
- Matriz OneHotEncoder

Veremos luego que este preprocesamiento manual es innecesario ya que muchas herramientas de tensorflow pueden hacerlo.


## Escalar las imágenes a 300x300

Para escalar todas la imágenes lo que haremos será tomas cadas una de las imágenes de la carpeta `images` y copiarlas en otra carpeta con el nombre `resized_images`. Por motivos de ahorro de CPU este y los demás métodos solo se aplicarán a una imagen de cada carpeta. 

Estas son las librerías que usaremos:

In [19]:
from PIL import Image, ImageEnhance
import numpy as np
import pandas as pd
import os
import random

In [20]:
def resize(input_path, output_path):
    dirs = os.listdir(input_path)
    for dirpath, dirnames, filenames in os.walk(input_path):
        if len(filenames) > 0:
            for i in range(5):
                filename = filenames[i]
                if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                    file_path = os.path.join(dirpath, filename)
                    
                    relative_dir = os.path.relpath(dirpath, input_path)
                    new_dir = os.path.join(output_path, relative_dir)
                    os.makedirs(new_dir, exist_ok=True)
                    
                    im = Image.open(file_path)
                    imResize = im.resize((300, 300))
                    
                    save_path = os.path.join(new_dir, f'{os.path.splitext(filename)[0]} resized.jpg')
                    imResize.save(save_path, 'JPEG', quality=90)
                    print(f'Imagen guardada en: {save_path}')

In [21]:
resize('images', 'resized_images')

Imagen guardada en: resized_images/Eugene_Delacroix/Eugene_Delacroix_16 resized.jpg
Imagen guardada en: resized_images/Eugene_Delacroix/Eugene_Delacroix_29 resized.jpg
Imagen guardada en: resized_images/Eugene_Delacroix/Eugene_Delacroix_14 resized.jpg
Imagen guardada en: resized_images/Eugene_Delacroix/Eugene_Delacroix_6 resized.jpg
Imagen guardada en: resized_images/Eugene_Delacroix/Eugene_Delacroix_9 resized.jpg
Imagen guardada en: resized_images/Claude_Monet/Claude_Monet_54 resized.jpg
Imagen guardada en: resized_images/Claude_Monet/Claude_Monet_23 resized.jpg
Imagen guardada en: resized_images/Claude_Monet/Claude_Monet_47 resized.jpg
Imagen guardada en: resized_images/Claude_Monet/Claude_Monet_51 resized.jpg
Imagen guardada en: resized_images/Claude_Monet/Claude_Monet_58 resized.jpg
Imagen guardada en: resized_images/Edvard_Munch/Edvard_Munch_41 resized.jpg
Imagen guardada en: resized_images/Edvard_Munch/Edvard_Munch_25 resized.jpg
Imagen guardada en: resized_images/Edvard_Munch/Ed

## Aumentar datos

Para aumentar datos a nuestro dataset lo que haremos será usar 5 formas para modificar las diferentes imágenes sin modificar su escencia. Cada uno de los siguiente modificadores aplica su efecto y aplitud del mismo de manera aleatoria, de tal manera que el modelo no crea que hay patrones en estos cambios.

### Flip horinzontal y vertical

Lo que hace esto es voltear la imagen como si se tratase de un reflejo de un espejo-

In [22]:
def random_flip(img):
    # Flip horizontal
    if random.random() > 0.5:
        img = img.transpose(Image.FLIP_LEFT_RIGHT)
    # Flip vertical
    if random.random() > 0.5:
        img = img.transpose(Image.FLIP_TOP_BOTTOM)
    return img


### Rotación

Esta si es la rotación común que gira la imagen respecto a su centro. 

In [23]:
def random_rotation(img, max_rotation=25):
    # Rotación aleatoria entre -max_rotation y max_rotation grados
    angle = random.uniform(-max_rotation, max_rotation)
    img = img.rotate(angle)
    return img

### Zoom

Acerca la imagen, y no necesariamente hacia el centro de la misma.

In [24]:
def random_zoom(img, zoom_factor=0.2):
    # Recorte y redimensionamiento para aplicar un efecto de zoom
    width, height = img.size
    crop_width = int(width * (1 - zoom_factor))
    crop_height = int(height * (1 - zoom_factor))
    
    left = random.randint(0, width - crop_width)
    upper = random.randint(0, height - crop_height)
    
    img_cropped = img.crop((left, upper, left + crop_width, upper + crop_height))
    img_resized = img_cropped.resize((width, height))
    
    return img_resized

### Translación

Mueve la imagen de manera que esta quede con otra centralización. Puede quedar más para arriba, abajo, derecha o izquierda.

In [25]:
def random_translation(img, max_tx=0.3, max_ty=0.3):
    # Traslación aleatoria de la imagen
    width, height = img.size
    tx = random.uniform(-max_tx, max_tx) * width
    ty = random.uniform(-max_ty, max_ty) * height
    
    img = img.transform(
        (width, height),
        Image.AFFINE,
        (1, 0, tx, 0, 1, ty),
        resample=Image.BICUBIC
    )
    return img

### Contraste

El contraste es la medida que indica que tan distanciados están unos tonos de otros. Tanto como el alto contraste como el bajo puede hacer que la imagen pierda información. Sin embargo, esto permitiria al modelo reconcer patrones fuera de los tonos usados en las pinturas.

In [26]:
def random_contrast(img, max_contrast=0.2):
    # Aumentar o disminuir el contraste de la imagen
    enhancer = ImageEnhance.Contrast(img)
    factor = random.uniform(1 - max_contrast, 1 + max_contrast)
    img = enhancer.enhance(factor)
    return img

### Aplicar los efectos de manera aleatoria

Conviene aplicar los distintos efectos de forma aleatoria ya que si no, se podría estar creando patrones que no queremos que influyan en el aprendizaje de nuestro modelo. De momento se generarán 5 imagenes por cada imagen pasada como parámetro

In [27]:
def generate_images_from_image(img, num_images=5):
    generated_images = [img]
    
    for _ in range(num_images - 1):
        new_img = img.copy()
        
        # Aplicar una combinación aleatoria de las transformaciones
        if random.random() > 0.5:
            new_img = random_flip(new_img)
        if random.random() > 0.5:
            new_img = random_rotation(new_img)
        if random.random() > 0.5:
            new_img = random_zoom(new_img)
        if random.random() > 0.5:
            new_img = random_translation(new_img)
        if random.random() > 0.5:
            new_img = random_contrast(new_img)
        
        generated_images.append(new_img)
    
    return generated_images

### Procesar dataset

In [28]:
def generate_images(input_path, output_path):
    dirs = os.listdir(input_path)
    for dirpath, dirnames, filenames in os.walk(input_path):
        if len(filenames) > 0:
            for filename in filenames:
                if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                    file_path = os.path.join(dirpath, filename)
                    
                    relative_dir = os.path.relpath(dirpath, input_path)
                    new_dir = os.path.join(output_path, relative_dir)
                    os.makedirs(new_dir, exist_ok=True)
                    
                    im = Image.open(file_path)
                    generate_images_list = generate_images_from_image(im)
                    
                    for i, generate_image in enumerate(generate_images_list):
                        
                        save_path = os.path.join(new_dir, f'{os.path.splitext(filename)[0]} {i} generated.jpg')
                        generate_image.save(save_path, 'JPEG', quality=90)
                        print(f'Imagen guardada en: {save_path}')

In [29]:
generate_images('resized_images', 'generated_images')

Imagen guardada en: generated_images/Eugene_Delacroix/Eugene_Delacroix_9 resized 0 generated.jpg
Imagen guardada en: generated_images/Eugene_Delacroix/Eugene_Delacroix_9 resized 1 generated.jpg
Imagen guardada en: generated_images/Eugene_Delacroix/Eugene_Delacroix_9 resized 2 generated.jpg
Imagen guardada en: generated_images/Eugene_Delacroix/Eugene_Delacroix_9 resized 3 generated.jpg
Imagen guardada en: generated_images/Eugene_Delacroix/Eugene_Delacroix_9 resized 4 generated.jpg
Imagen guardada en: generated_images/Eugene_Delacroix/Eugene_Delacroix_6 resized 0 generated.jpg
Imagen guardada en: generated_images/Eugene_Delacroix/Eugene_Delacroix_6 resized 1 generated.jpg
Imagen guardada en: generated_images/Eugene_Delacroix/Eugene_Delacroix_6 resized 2 generated.jpg
Imagen guardada en: generated_images/Eugene_Delacroix/Eugene_Delacroix_6 resized 3 generated.jpg
Imagen guardada en: generated_images/Eugene_Delacroix/Eugene_Delacroix_6 resized 4 generated.jpg
Imagen guardada en: generated_

## Normalizar los datos de 0 a 1

Muchos modelo funciona mejor cuando los datos están normalizados. Ya sea de -1 a 1, o de 0 a 1 (lo más común) los datos normalizados ayudan a que estos modelos puedan trabajar de manera más eficiente con los datos, si que estos pierdan la información relevante. Mismo caso que el anterior: Por motivos de costos computacionales, solo se trabajará sobre una imagen por carpeta. Hay que tomar en cuenta que muchas imágenes no se pueden guardar con valores de entre 0 y 1, si no, valores enteros de entre 0 y 255. Por lo tanto guardaremos las imágenes normalizadas como arreglos de numpy.

In [30]:
def normalize_and_resize(input_path, output_path):
    dirs = os.listdir(input_path)
    for dirpath, dirnames, filenames in os.walk(input_path):
        if len(filenames) > 0:
            filename = filenames[0]
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                file_path = os.path.join(dirpath, filename)
                
                relative_dir = os.path.relpath(dirpath, input_path)
                new_dir = os.path.join(output_path, relative_dir)
                os.makedirs(new_dir, exist_ok=True)
                
                im = Image.open(file_path)
                img_array = np.array(im)
                
                img_normalized = img_array / 255.0
                
                npy_save_path = os.path.join(new_dir, f'{os.path.splitext(filename)[0]} normalized.npy')
                np.save(npy_save_path, img_normalized)
                print(f'Arreglo guardado en: {npy_save_path}')

In [31]:
normalize_and_resize('generated_images', 'normalized_images')

Arreglo guardado en: normalized_images/Eugene_Delacroix/Eugene_Delacroix_9 resized 1 generated normalized.npy
Arreglo guardado en: normalized_images/Claude_Monet/Claude_Monet_54 resized 3 generated normalized.npy
Arreglo guardado en: normalized_images/Edvard_Munch/Edvard_Munch_32 resized 1 generated normalized.npy
Arreglo guardado en: normalized_images/Sandro_Botticelli/Sandro_Botticelli_159 resized 2 generated normalized.npy
Arreglo guardado en: normalized_images/Paul_Klee/Paul_Klee_15 resized 4 generated normalized.npy
Arreglo guardado en: normalized_images/Albrecht_Du╠êrer/Albrecht_Du╠êrer_180 resized 4 generated normalized.npy
Arreglo guardado en: normalized_images/Georges_Seurat/Georges_Seurat_34 resized 4 generated normalized.npy
Arreglo guardado en: normalized_images/Jackson_Pollock/Jackson_Pollock_13 resized 2 generated normalized.npy
Arreglo guardado en: normalized_images/Edgar_Degas/Edgar_Degas_1 resized 3 generated normalized.npy
Arreglo guardado en: normalized_images/Rene_M

## Matriz de One Hot Encoder

In [32]:

def one_hot_encoder_labels(input_path):
    dirs = sorted([d for d in os.listdir(input_path) if os.path.isdir(os.path.join(input_path, d))])
    
    class_to_index = {class_name: idx for idx, class_name in enumerate(dirs)}
    
    data = []
    
    for dir_name in dirs:
        folder_path = os.path.join(input_path, dir_name)
        for filename in os.listdir(folder_path):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                one_hot = [0] * len(dirs)
                one_hot[class_to_index[dir_name]] = 1
                data.append({'image': filename, 'one_hot': one_hot})
    
    df = pd.DataFrame(data)
    return df

In [33]:
df = one_hot_encoder_labels('images')
print(df)

                        image  \
0      Albrecht_Dürer_43.jpg   
1     Albrecht_Dürer_153.jpg   
2     Albrecht_Dürer_180.jpg   
3      Albrecht_Dürer_29.jpg   
4     Albrecht_Dürer_237.jpg   
...                       ...   
8769    William_Turner_11.jpg   
8770    William_Turner_36.jpg   
8771    William_Turner_28.jpg   
8772    William_Turner_55.jpg   
8773    William_Turner_49.jpg   

                                                one_hot  
0     [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
1     [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
2     [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
3     [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
4     [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
...                                                 ...  
8769  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
8770  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
8771  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
8772  [0, 0, 0, 0, 0, 0

Mucho de lo que hicimos anteriormente no es necesario, ya que Tensorflow puede realizar estos proprocesamientos de manera mucho más sencilla.